# Setup - Environment Configuration

**Objective:** Configure an isolated environment for each training participant.

**Scope:**
- User detection and unique schema creation
- Unity Catalog configuration
- Utility functions (environment reset)

---

## Context and Requirements

- **Training day:** All days - required before each Demo notebook
- **Notebook type:** Setup (invoked via `%run ./00_Setup`)
- **Technical requirements:**
  - Databricks Runtime 14.x LTS or newer
  - Unity Catalog enabled
  - Permissions: CREATE CATALOG, CREATE SCHEMA
- **Execution time:** < 1 minute

## Section 1: User Detection

Automatic detection of current user and schema name preparation:

In [0]:
# Get current user email and extract username
current_user_email = spark.sql("SELECT current_user()").collect()[0][0]
username = current_user_email.split("@")[0].replace(".", "_").replace("-", "_")

print(f"Detected user: {current_user_email}")
print(f"Username for schema: {username}")

## Section 2: Catalog and Schema Configuration

Definition of target Unity Catalog and unique per-user schema:

In [0]:
# Configuration
if "trainer" in username or "krzysztof_burejza" in username:
    effective_user = "trainer"
else:
    effective_user = username

catalog_name = "data_ml_preparation" # Ensure this catalog exists or use 'hive_metastore' if UC is not enabled
schema_name = f"ml_dp_{effective_user}"

print(f"Target Catalog: {catalog_name}")
print(f"Target Schema: {schema_name}")

## Section 3: Environment Creation

Create catalog and schema in Unity Catalog:

In [0]:
# Create and Set Schema

#spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"USE CATALOG {catalog_name}")
    
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
spark.sql(f"USE SCHEMA {schema_name}")

print(f"✅ Successfully configured environment.")

## Section 4: User Context

Display current context (for verification):

In [0]:
# Display current context for verification
display(
    spark.createDataFrame([
        ("CATALOG", catalog_name),
        ("SCHEMA", schema_name),
        ("USER", username),
        ("FULL_PATH", f"{catalog_name}.{schema_name}")
    ], ["Variable", "Value"])
)

## Section 5: Utility Functions

Function to reset environment (optional - use when you want to start fresh):

In [0]:

# Utility function to reset environment (Optional)
def reset_environment():
    spark.sql(f"DROP SCHEMA IF EXISTS {schema_name} CASCADE")
    spark.sql(f"CREATE SCHEMA {schema_name}")
    print(f"Schema {schema_name} has been reset.")